In [1]:
import re
import pandas as pd

In [2]:
dataset_full_paths = !find . -name '*.csv'
dataset_full_paths

['./datasets_combinacoes/motodroid/SVM/svm_output_sigpid_sigapi_jowmdroid.csv',
 './datasets_combinacoes/motodroid/SVM/svm_output_sigpid_rfg.csv',
 './datasets_combinacoes/motodroid/SVM/svm_output_rl_sigapi_jowmdroid.csv',
 './datasets_combinacoes/motodroid/SVM/svm_output_rl_sigapi.csv',
 './datasets_combinacoes/motodroid/SVM/svm_output_rl_rfg.csv',
 './datasets_combinacoes/motodroid/SVM/svm_output_sigpid_rfg_jowmdroid.csv',
 './datasets_combinacoes/motodroid/SVM/svm_output_rl_rfg_jowmdroid.csv',
 './datasets_combinacoes/motodroid/SVM/svm_output_sigpid_sigapi.csv',
 './datasets_combinacoes/motodroid/RF/rf_output_sigpid_sigapi_jowmdroid.csv',
 './datasets_combinacoes/motodroid/RF/rf_output_sigpid_rfg_jowmdroid.csv',
 './datasets_combinacoes/motodroid/RF/rf_output_rl_rfg.csv',
 './datasets_combinacoes/motodroid/RF/rf_output_rl_rfg_jowmdroid.csv',
 './datasets_combinacoes/motodroid/RF/rf_output_rl_sigapi_jowmdroid.csv',
 './datasets_combinacoes/motodroid/RF/rf_output_rl_sigapi.csv',
 './d

In [3]:
possible_methods = [
    # alguns arquivos possuem a sigla em inglês (LR); e outros, em português (RL)
    'LR',
    'RL',
    
    'SigAPI',
    'RFG',
    'SigPID',
    'JOWMDroid'
]

table = []
for full_path in dataset_full_paths:
    data = pd.read_csv(full_path)    

    filename = full_path.split('/')[-1]
    model_name = 'RF' if filename[:9] == 'rf_output' else 'SVM'
    data['Modelo'] = model_name
    
    methods_used = []
    for method in possible_methods:
        method_name = 'LR' if method == 'RL' else method
        column_name = 'Método ' + method_name
        is_method_used = method.lower() in '/'.join(full_path.split('/')[2:]).lower()
        if(is_method_used):
            methods_used.append(method_name)
            data[column_name] = 1
        else:
            data[column_name] = 0
            
    
    # transforma o nome do dataset de algo como
    # 'svm_output_resultado_rl_sigapi_jowmdroid.csv.csv'
    # para
    # 'lr_sigapi_jowmdroid'
    dataset_name = filename.replace('.csv.csv', '.csv')
    dataset_name = re.sub('((rf_|svm_)output_|resultado_)', '', dataset_name)
    dataset_name = re.sub('(_)*rl(_)*', '_lr_', dataset_name)
    dataset_name = re.sub('(^_|_$)', '', dataset_name)
    data['Dataset'] = dataset_name.replace('.csv', '')
    
    data['Combinação'] = 'ORIGINAL' if len(methods_used) == 0 else ' + '.join(methods_used)
    table.append(data)

df = pd.concat(table)

In [7]:
df

,Accuracy,Precision,Recall,F1_Score,RoC_AuC,Modelo,Método LR,Método SigAPI,Método RFG,Método SigPID,Método JOWMDroid,Dataset,Combinação
0,95.247482,71.716470,36.299683,48.201775,67.685516,SVM,0,1,0,1,1,sigpid_sigapi_jowmdroid,SigAPI + SigPID + JOWMDroid
0,96.147418,74.404484,56.032360,63.924559,77.390995,SVM,0,0,1,1,0,sigpid_rfg,RFG + SigPID
0,95.206771,69.523196,37.952867,49.101251,68.436812,SVM,1,1,0,0,1,lr_sigapi_jowmdroid,LR + SigAPI + JOWMDroid
0,95.088922,67.165109,37.917693,48.471223,68.357619,SVM,1,1,0,0,0,lr_sigapi,LR + SigAPI
0,95.988858,71.154684,57.439325,63.565590,77.964420,SVM,1,0,1,0,0,lr_rfg,LR + RFG
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,93.914720,56.521739,0.457263,0.907188,50.217223,RF,0,0,0,1,0,sigpid_motodroid_intents,SigPID
0,95.249817,95.955952,90.981098,93.402329,94.366657,RF,0,0,0,1,0,sigpid_drebin_215_api_calls_limpo,SigPID
0,93.908292,0.000000,0.000000,0.000000,50.000000,RF,0,0,0,1,0,sigpid_motodroid_all,SigPID
0,93.908292,0.000000,0.000000,0.000000,50.000000,RF,0,0,0,1,0,sigpid_motodroid_permissions,SigPID
